In [1]:
import torch

from ppuu.modeling.km import predict_states, predict_states_seq
from ppuu.data.dataloader import Normalizer

bsize = 1
n_actions = 10

states = torch.randn(bsize, 5)
actions = torch.randn(bsize, n_actions, 2)

normalizer = Normalizer.dummy()

def naive(states, actions, normalizer):
    predictions = []
    for i in range(actions.shape[1]):
        states = predict_states(
            states,
            actions[:, i],
            normalizer,
        )
        predictions.append(states)
    return torch.stack(predictions, dim=1)

s1 = naive(states, actions, normalizer)
s2 = predict_states_seq(states, actions, normalizer)

In [2]:
import torch
from ppuu.data.dataloader import Dataset, DataStore
from torch.utils.data import DataLoader

# ds_path = "/misc/vlgscratch4/LecunGroup/nvidia-collab/traffic-data_offroad/state-action-cost/data_i80_v0/"
# ds_path = "/misc/vlgscratch4/LecunGroup/nvidia-collab/vlad/traffic-data_offroad_50_test_train_same/state-action-cost/data_i80_v0/"
ds_path = "/home/us441/nvidia-collab/vlad/traffic-data-5-small/state-action-cost/data_i80_v0/"

data_store = DataStore(ds_path)

dataset = Dataset(
    data_store, "val", 20, 30, size=1000, normalize=True, shift=False, random_actions=False,
)

loader = DataLoader(
    dataset,
    batch_size=6,
    num_workers=0,
)


batch = next(iter(loader))

In [3]:
from ppuu.lightning_modules.fm import FM

m_config = FM.Config()
m_config.model.fm_type = "km_no_action"
m_config.model.checkpoint = "/home/us441/nvidia-collab/vlad/results/fm/km_no_action/fm_km_no_action_64/seed=42/checkpoints/last.ckpt"
m_config.training.enable_latent = True
m_config.training.diffs = False
forward_model = FM(m_config).cuda()
forward_model._setup_normalizer(data_store.stats)

In [4]:
batch.sample_split_id

('/home/us441/nvidia-collab/vlad/traffic-data-5/data_i80_v0//trajectories-0400-0415/car1025.pkl',
 '/home/us441/nvidia-collab/vlad/traffic-data-5/data_i80_v0//trajectories-0400-0415/car100.pkl',
 '/home/us441/nvidia-collab/vlad/traffic-data-5/data_i80_v0//trajectories-0400-0415/car103.pkl',
 '/home/us441/nvidia-collab/vlad/traffic-data-5/data_i80_v0//trajectories-0400-0415/car1045.pkl',
 '/home/us441/nvidia-collab/vlad/traffic-data-5/data_i80_v0//trajectories-0400-0415/car1013.pkl',
 '/home/us441/nvidia-collab/vlad/traffic-data-5/data_i80_v0//trajectories-0400-0415/car1036.pkl')

In [7]:
from ppuu.data.entities import DatasetSample

b = batch.cuda()
print(b)

DatasetSample(conditional_state_seq=StateSequence(images=tensor([[[[[0.0000, 0.0000, 0.0000,  ..., 0.3765, 0.6275, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.3765, 0.6275, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.3765, 0.6275, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.8784, 0.1412],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.8784, 0.1412],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.8784, 0.1412]],

          [[0.7059, 0.0902, 0.0000,  ..., 0.1529, 0.8902, 1.0000],
           [0.0902, 0.0078, 0.0000,  ..., 0.1255, 0.8745, 1.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.1255, 0.8745, 1.0000],
           ...,
           [0.9647, 0.2824, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.9647, 0.2824, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.9647, 0.2824, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000, 

In [8]:
predictions = forward_model.model.unfold(b.conditional_state_seq, b.target_action_seq)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(dpi=200)
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(predictions.images[1, i*3 + 2].detach().cpu().permute(1, 2, 0))
    plt.axis('off')

In [8]:
%debug

> /home/us441/vlad/miniconda3/envs/forked/lib/python3.8/site-packages/torch/nn/functional.py(1610)linear()
   1608     if input.dim() == 2 and bias is not None:
   1609         # fused op is marginally faster
-> 1610         ret = torch.addmm(bias, input, weight.t())
   1611     else:
   1612         output = input.matmul(weight.t())

ipdb> up
> /home/us441/vlad/miniconda3/envs/forked/lib/python3.8/site-packages/torch/nn/modules/linear.py(87)forward()
     85 
     86     def forward(self, input):
---> 87         return F.linear(input, self.weight, self.bias)
     88 
     89     def extra_repr(self):

ipdb> up
> /home/us441/vlad/miniconda3/envs/forked/lib/python3.8/site-packages/torch/nn/modules/module.py(550)__call__()
    548             result = self._slow_forward(*input, **kwargs)
    549         else:
--> 550             result = self.forward(*input, **kwargs)
    551         for hook in self._forward_hooks.values():
    552             hook_result = hook(self, input, result)

ip

ipdb> z_val.device
device(type='cpu')
ipdb> type(z)
<class 'torch.Tensor'>
ipdb> z.device
device(type='cpu')
ipdb> q


In [ ]:
import time
def test(f, n_iter=20):
    start = time.time()

    n_iter = 20
    
    for i in range(n_iter):
        f()

    print((time.time() - start) / n_iter)

In [ ]:
test(f)

In [ ]:
from ppuu.modeling.km import predict_states, predict_states_seq


def kmf():
    actions = torch.zeros_like(batch['actions'])
    actions.requires_grad = True
    pred = predict_states_seq(batch['input_states'][:, -1], actions, forward_model.normalizer) 
    return pred
    pred.sum().backward()
    
def kmf_grad():
    pred = kmf()
    pred[:, :, 0].sum().backward()

def kmf_cpu():
    actions = torch.zeros_like(batch['actions'])
    actions.requires_grad = True
    pred = predict_states_seq(batch['input_states'][:, -1].cpu(), actions.cpu(), forward_model.normalizer) 
    return pred
    
def kmf_cpu_grad():
    pred = kmf_cpu()
    pred[:, :, 0].sum().backward()

In [ ]:
test(kmf)
test(kmf_grad)
test(kmf_cpu)
test(kmf_cpu_grad)

In [1]:
import torch

In [2]:
x = torch.tensor([1, 2, 3])

In [3]:
x

tensor([1, 2, 3])

In [9]:
y = x.unsqueeze(0).expand(3, 3).contiguous()

In [10]:
y.view(9)

tensor([1, 2, 3, 1, 2, 3, 1, 2, 3])

In [19]:
x.repeat(3)

tensor([1, 2, 3, 1, 2, 3, 1, 2, 3])